In [1]:
import pandas as pd
import numpy as np

# 데이터 로드
df = pd.read_excel('funddata3.xlsx')

# 연환산 수익률 계산 함수
def annualized_return(cumulative_return, periods, period_type='year'):
    if period_type == 'month':
        n = periods / 12
    elif period_type == 'year':
        n = periods
    else:
        raise ValueError("period_type은 'month' 또는 'year'이어야 합니다.")

    if n <= 0:
        return 0
    return ((1 + cumulative_return / 100) ** (1 / n) - 1) * 100

# 수익률 선택 함수
def get_selected_return(row, investment_period):
    if investment_period <= 1:
        # 1년 이하 투자자는 6개월 수익률 사용
        if not pd.isna(row['6개월수익률']):
            return row['6개월수익률']
        elif not pd.isna(row['1년수익률']):
            return row['1년수익률']
    elif 1 < investment_period <= 3:
        # 1년 초과 ~ 3년 이하 투자자는 1년 수익률 사용
        if not pd.isna(row['1년수익률']):
            return row['1년수익률']
        elif not pd.isna(row['6개월수익률']):
            return row['6개월수익률']
    else:
        # 3년 초과 투자자는 3년 수익률 사용
        if not pd.isna(row['3년수익률']):
            return row['3년수익률']
        elif not pd.isna(row['1년수익률']):
            return row['1년수익률']
    # 필요한 수익률이 없을 경우 NaN 반환
    return np.nan

# 고객 프로파일 정의
customer_profiles = {
    1: {
        'risk_tolerance': 'low',
        'investment_period': 1,
        'expected_return': 'market_return',
        'weights': {'risk_score': 0.6, 'return_score': 0.25, 'fee_score': 0.15},
        'preferred_fund_types': ['기타', 'ETF', '채권형', 'MMF']
    },
    2: {
        'risk_tolerance': 'medium',
        'investment_period': 3,
        'expected_return': 'high_return',
        'weights': {'risk_score': 0.2, 'return_score': 0.7, 'fee_score': 0.1},
        'preferred_fund_types': ['기타', 'ETF', '주식형']
    },
}

# 위험 점수 계산 (1등급 -> 0점, 6등급 -> 100점)
df['risk_score'] = (df['펀드위험'] - 1) / (6 - 1) * 100

# 수수료 점수 계산 (수수료가 낮을수록 높은 점수)
if df['수수료'].max() != df['수수료'].min():
    df['fee_score'] = (df['수수료'].max() - df['수수료']) / (df['수수료'].max() - df['수수료'].min()) * 100
else:
    df['fee_score'] = 100  # 모든 펀드의 수수료가 동일한 경우 모든 펀드에 동일한 점수 부여

# 펀드 추천 함수
def recommend_funds(df, profile_id):
    profile = customer_profiles[profile_id]
    temp_df = df.copy()

    # 펀드종류를 소문자로 변환하여 표준화
    temp_df['펀드종류'] = temp_df['펀드종류'].str.lower().str.strip()
    preferred_types = [ptype.lower() for ptype in profile['preferred_fund_types']]

    # 선호 펀드종류로 필터링
    temp_df = temp_df[temp_df['펀드종류'].isin(preferred_types)]

    # 투자 기간에 따른 수익률 선택 및 점수 계산
    investment_period = profile['investment_period']
    temp_df['selected_return'] = temp_df.apply(lambda row: get_selected_return(row, investment_period), axis=1)

    # 선택된 수익률이 없는 경우 제외
    temp_df = temp_df.dropna(subset=['selected_return'])

    if temp_df.empty:
        return pd.DataFrame()  # Empty DataFrame

    # 선택된 수익률을 기반으로 점수화 (0~100)
    min_return = temp_df['selected_return'].min()
    max_return = temp_df['selected_return'].max()
    if max_return - min_return != 0:
        temp_df['return_score'] = (temp_df['selected_return'] - min_return) / (max_return - min_return) * 100
    else:
        temp_df['return_score'] = 50  # 모든 수익률이 동일한 경우 중간 점수 부여

    # 최종 점수 계산
    temp_df['total_score'] = (
        temp_df['risk_score'] * profile['weights']['risk_score'] +
        temp_df['return_score'] * profile['weights']['return_score'] +
        temp_df['fee_score'] * profile['weights']['fee_score']
    )

    # 점수 순으로 정렬하여 상위 펀드 반환
    recommended_funds = temp_df.sort_values(by='total_score', ascending=False)

    # 필요한 열만 선택
    recommended_funds = recommended_funds[['펀드종류', '펀드명', 'total_score', 'selected_return']]

    return recommended_funds

# 펀드 추천 및 저장을 위한 딕셔너리 초기화
recommendations = {}

# 펀드 추천 예시
for profile_id in customer_profiles.keys():
    recommended_funds = recommend_funds(df, profile_id)
    if not recommended_funds.empty:
        display_df = recommended_funds.head(10).reset_index(drop=True)
        print(f"프로파일 {profile_id} 추천 펀드:")
        print(display_df)
        print("\n")
        # 추천 펀드를 딕셔너리에 저장
        recommendations[f'Profile_{profile_id}'] = display_df
    else:
        print(f"프로파일 {profile_id} 에 대한 추천 펀드가 없습니다.")
        print("\n")
        # 추천 펀드가 없을 경우, 메시지 포함
        recommendations[f'Profile_{profile_id}'] = pd.DataFrame({
            '메시지': ['추천 펀드가 없습니다.']
        })

# 엑셀로 저장
with pd.ExcelWriter('recommended_funds.xlsx') as writer:
    for profile_name, df_recommend in recommendations.items():
        # 엑셀 시트 이름은 최대 31자까지 허용됨
        sheet_name = profile_name[:31]
        df_recommend.to_excel(writer, sheet_name=sheet_name, index=False)

print("추천 펀드가 'recommended_funds.xlsx' 파일로 저장되었습니다.")


프로파일 1 추천 펀드:
  펀드종류                                         펀드명  total_score  \
0  etf                  KBRISE금융채액티브증권상장지수투자신탁(채권)    82.225964   
1  etf  미래에셋TIGERCD금리투자KIS특별자산상장지수투자신탁(CD-파생형)(합성)    82.109457   
2  etf                   키움KOSEF단기자금증권상장지수투자신탁[채권]    82.020939   
3  etf                KBRISE단기국공채액티브증권상장지수투자신탁(채권)    81.956884   
4  etf                   한국투자ACE단기자금증권상장지수투자신탁(채권)    81.929364   
5  etf                 한화PLUS단기채권액티브증권상장지수투자신탁(채권)    81.920191   
6  etf              미래에셋TIGER단기채권액티브증권상장지수투자신탁(채권)    81.914075   
7  etf                 신한SOLKIS단기통안채증권상장지수투자신탁[채권]    81.886556   
8  etf                미래에셋TIGER단기통안채증권상장지수투자신탁(채권)    81.794981   
9  etf                 한화PLUS우량회사채50증권상장지수투자신탁(채권)    81.689841   

   selected_return  
0           0.0284  
1           0.0178  
2           0.0183  
3           0.0196  
4           0.0187  
5           0.0184  
6           0.0182  
7           0.0173  
8           0.0177  
9           0.0219  


프로파일 2 추천 펀드:
 